In [1]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN

import matplotlib.pyplot as plt

import seaborn as sns

import pandas as pd
from datetime import datetime, timedelta

import seaborn as sns

from matplotlib import rcParams
from sklearn.linear_model import LinearRegression

from collections import Counter

sns.set(rc={'figure.figsize': (15.7, 13.27)})
rcParams['figure.figsize'] = 15.7,13.27
plt.rcParams['font.family'] = 'AppleGothic'

xlsx = pd.read_excel('datas.xlsx', 
                    header = None, 
                    skiprows = 2,
                    engine = 'openpyxl')

data_startcol = 7
ogDatasObj = {}
for col in xlsx:
    if col >= data_startcol: # startcol 7
        ogDatasObj[f'{xlsx[col][0]}-{xlsx[col][1]}-{xlsx[col][2]}'] = xlsx[col][3:]

ogDatas = pd.DataFrame(ogDatasObj)
ogDatas = ogDatas.fillna(0)
ogDatas = ogDatas.reset_index()
del ogDatas['index']

pd.set_option('display.max_row', 20)
pd.set_option('display.max_columns', 100)

ogDatas

ogDatas.replace('-', 0, inplace=True)
uids = ogDatas.columns

uids

SEASONS = {
    "봄" : ["03", "04", "05"],
    "여름" : ["06", "07", "08"],
    "가을" : ["09", "10", "11"],
    "겨울" : ["12", "01", "02"]
}

SEASONSARR = ['봄','여름','가을','겨울']

'''
 0 : 봄
 1 : 여름
 2 : 가을
 3 : 겨울
'''

# 예제 데이터 준비
startDate = datetime(2018,5,1)
endDate = datetime(2019,5,1)
def datetime_range(start, end, delta):
    current = start
    while current < end:
        yield current
        current += delta
dateIndexArr = [dt.strftime('%Y%m%d') for dt in
                   datetime_range(startDate, endDate, timedelta(days = 1))]
# 아파트4-103-606
searchKeyword = '아파트4-103-606'
tsDatasObj = {}
startIdx = 0;
endIdx = 96;
for date in dateIndexArr:
    tsDatasObj[date] = ogDatas[searchKeyword][startIdx:endIdx].values.tolist()
    startIdx = endIdx
    endIdx = endIdx + 96

tsDatas = pd.DataFrame(tsDatasObj)
tsDatas = tsDatas.T
tsDatas = tsDatas.reset_index()
tsDatas.rename(columns={'index':'date'}, inplace=True)
tsDatas['year'] = [dt[0:4] for dt in tsDatas['date']]
tsDatas['month'] = [dt[4:6] for dt in tsDatas['date']]
tsDatas['day'] = [dt[6:8] for dt in tsDatas['date']]
tsDatas['date'] = [datetime.strptime(dt, "%Y%m%d") for dt in tsDatas['date']]
tsDatas.reindex(columns=['date', 'year', 'month', 'day'])

datas = [
        tsDatas[(tsDatas['month'].isin(SEASONS[season]))].copy()
        for season in ["봄", "여름", "가을", "겨울"]
    ]

datas[0] # 봄 데이터
datas[1] # 여름 데이터
datas[2] # 가을 데이터
datas[3] # 겨울 데이터

,date,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,year,month,day
214,2018-12-01,0.112,0.121,0.116,0.122,0.114,0.108,0.076,0.071,0.073,0.057,0.100,0.079,0.061,0.076,0.064,0.053,0.066,0.064,0.080,0.066,0.054,0.062,0.064,0.073,0.074,0.096,0.056,0.055,0.058,0.049,0.044,0.032,0.049,0.065,0.048,0.045,0.036,0.058,0.083,0.101,0.127,0.088,0.126,0.120,0.099,0.073,0.062,0.078,0.063,0.066,0.055,0.054,0.071,0.100,0.089,0.084,0.061,0.058,0.069,0.092,0.100,0.059,0.056,0.078,0.068,0.061,0.063,0.055,0.094,0.069,0.080,0.076,0.073,0.089,0.101,0.098,0.072,0.097,0.074,0.087,0.063,0.065,0.057,0.045,0.053,0.065,0.066,0.055,0.054,0.086,0.108,0.074,0.093,0.098,0.113,0.104,2018,12,01
215,2018-12-02,0.039,0.036,0.037,0.050,0.055,0.044,0.036,0.038,0.036,0.034,0.040,0.046,0.040,0.037,0.028,0.048,0.070,0.043,0.045,0.027,0.038,0.039,0.042,0.038,0.035,0.038,0.036,0.035,0.041,0.026,0.049,0.035,0.039,0.060,0.052,0.037,0.035,0.039,0.034,0.032,0.041,0.040,0.045,0.037,0.032,0.061,0.066,0.086,0.058,0.067,0.110,0.577,0.184,0.158,0.225,0.163,0.138,0.096,0.119,0.056,0.031,0.048,0.054,0.048,0.039,0.041,0.038,0.052,0.072,0.098,0.065,0.066,0.067,0.054,0.050,0.049,0.051,0.074,0.296,0.240,0.127,0.132,0.116,0.121,0.123,0.124,0.217,0.134,0.113,0.096,0.122,0.143,0.108,0.119,0.113,0.118,2018,12,02
216,2018-12-03,0.148,0.111,0.136,0.053,0.050,0.060,0.031,0.086,0.061,0.055,0.049,0.044,0.043,0.043,0.057,0.050,0.053,0.048,0.044,0.056,0.045,0.042,0.037,0.059,0.075,0.077,0.043,0.108,0.046,0.035,0.036,0.038,0.040,0.048,0.046,0.037,0.059,0.047,0.087,0.104,0.094,0.039,0.072,0.052,0.032,0.040,0.063,0.069,0.081,0.076,0.061,0.052,0.115,0.158,0.074,0.052,0.050,0.049,0.061,0.073,0.078,0.069,0.069,0.063,0.101,0.132,0.119,0.123,0.299,0.277,0.188,0.178,0.139,0.124,0.109,0.125,0.115,0.111,0.117,0.121,0.124,0.114,0.101,0.111,0.135,0.147,0.121,0.148,0.145,0.110,0.112,0.141,0.126,0.057,0.059,0.054,2018,12,03
217,2018-12-04,0.071,0.044,0.061,0.058,0.063,0.069,0.057,0.052,0.048,0.057,0.051,0.050,0.050,0.050,0.060,0.045,0.044,0.040,0.046,0.062,0.050,0.053,0.048,0.051,0.050,0.039,0.047,0.051,0.056,0.053,0.088,0.111,0.112,0.089,0.087,0.067,0.047,0.049,0.099,0.088,0.040,0.040,0.038,0.052,0.040,0.042,0.033,0.116,0.079,0.088,0.330,0.086,0.092,0.108,0.105,0.122,0.176,0.103,0.096,0.127,0.118,0.142,0.165,0.164,0.145,0.132,0.118,0.113,0.122,0.105,0.112,0.113,0.099,0.097,0.101,0.116,0.368,0.194,0.113,0.117,0.121,0.121,0.123,0.117,0.106,0.088,0.089,0.082,0.065,0.060,0.055,0.058,0.058,0.107,0.097,0.060,2018,12,04
218,2018-12-05,0.054,0.062,0.059,0.052,0.059,0.049,0.055,0.056,0.042,0.042,0.053,0.055,0.049,0.042,0.068,0.089,0.045,0.036,0.047,0.057,0.046,0.046,0.043,0.045,0.050,0.049,0.043,0.042,0.054,0.047,0.039,0.056,0.090,0.087,0.058,0.052,0.038,0.056,0.075,0.124,0.093,0.045,0.052,0.059,0.050,0.046,0.048,0.063,0.080,0.100,0.093,0.049,0.042,0.116,0.097,0.042,0.050,0.063,0.058,0.051,0.112,0.129,0.528,0.188,0.126,0.102,0.088,0.092,0.066,0.047,0.044,0.044,0.091,0.093,0.102,0.118,0.111,0.120,0.106,0.112,0.106,0.112,0.147,0.151,0.157,0.148,0.144,0.127,0.099,0.126,0.123,0.109,0.115,0.104,0.104,0.108,2018,12,05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,2019-02-24,0.107,0.107,0.092,0.093,0.096,0.098,0.096,0.078,0.046,0.050,0.046,0.058,0.037,0.049,0.056,0.044,0.057,0.045,0.037,0.059,0.052,0.047,0.048,0.047,0.083,0.058,0.048,0.045,0.035,0.059,0.055,0.053,0.044,0.035,0.060,0.054,0.053,0.086,0.078,0.095,0.049,0.115,0.139,0.091,0.067,0.

In [2]:
ogDatas

,아파트1-104-1206,아파트1-104-303,아파트1-104-1307,아파트1-104-1208,아파트1-104-408,아파트1-104-203,아파트1-103-1402,아파트1-103-402,아파트1-103-1201,아파트1-103-801,아파트1-103-502,아파트1-103-1602,아파트1-103-301,아파트1-103-1502,아파트1-103-202,아파트1-102-903,아파트1-102-603,아파트1-102-203,아파트1-102-704,아파트1-102-703,아파트1-102-403,아파트1-102-303,아파트1-101-604,아파트1-101-1602,아파트1-101-1002,아파트1-101-1202,아파트1-101-1604,아파트1-101-402,아파트1-101-303,아파트1-101-502,아파트1-101-1403,아파트1-101-904,아파트1-101-1502,아파트1-101-1102,아파트1-101-901,아파트1-101-704,아파트1-101-504,아파트1-102-104,아파트1-102-301,아파트1-102-1501,아파트1-102-1602,아파트1-102-1002,아파트1-102-401,아파트1-102-802,아파트1-102-801,아파트1-103-505,아파트1-103-1805,아파트1-103-1306,아파트1-103-1806,아파트1-103-803,...,아파트4-103-609,아파트4-103-707,아파트4-103-101,아파트4-102-803,아파트4-101-1501,아파트4-103-404,아파트4-103-909,아파트4-101-1203,아파트4-103-806,아파트4-103-1410,아파트4-103-402,아파트4-103-703,아파트4-103-106,아파트4-103-910,아파트4-102-402,아파트4-103-907,아파트4-103-1406,아파트4-103-1006,아파트4-103-608,아파트4-103-403,아파트4-105-1102,아파트4-103-804,아파트4-103-103,아파트4-103-706,아파트4-104-1001,아파트4-105-103,아파트4-102-401,아파트4-103-1104,아파트4-103-906,아파트4-103-606,아파트4-103-1307,아파트4-103-410,아파트4-103-1309,아파트4-102-102,아파트4-103-508,아파트4-105-1104,아파트4-102-1402,아파트4-103-407,아파트4-103-1206,아파트4-103-108,아파트4-105-102,아파트4-103-1102,아파트4-105-601,아파트4-104-1501,아파트4-103-702,아파트4-102-104,아파트4-103-902,아파트4-101-602,아파트4-103-905,아파트4-101-104
0,0.085,0.016,0.052,0.003,0.032,0.074,0.053,0.096,0.105,0.042,0.149,0.039,0.054,0.132,0.091,0.169,0.051,0.118,0.060,0.047,0.047,0.053,0.046,0.104,0.079,0.092,0.043,0.069,0.139,0.026,0.146,0.092,0.025,0.127,0.045,0.042,0.113,0.207,0.030,0.171,0.045,0.064,0.045,0.277,0.186,0.142,0.132,0.034,0.088,0.041,...,0.041,0.137,0.114,0.046,0.098,0.058,0.068,0.040,0.063,0.048,0.168,0.053,0.099,0.075,0.080,0.158,0.024,0.153,0.056,0.037,0.091,0.069,0.128,0.092,0.072,0.069,0.034,0.166,0.086,0.113,0.184,0.175,0.096,0.054,0.080,0.084,0.119,0.088,0.117,0.161,0.182,0.044,0.051,0.057,0.126,0.094,0.065,0.116,0.093,0.147
1,0.086,0.049,0.053,0.002,0.022,0.014,0.048,0.090,0.108,0.022,0.119,0.023,0.075,0.142,0.063,0.183,0.045,0.164,0.065,0.066,0.043,0.056,0.045,0.130,0.081,0.102,0.042,0.083,0.176,0.046,0.113,0.088,0.026,0.116,0.066,0.050,0.139,0.188,0.031,0.165,0.032,0.068,0.052,0.244,0.209,0.205,0.161,0.039,0.094,0.036,...,0.036,0.083,0.124,0.055,0.115,0.068,0.072,0.032,0.085,0.111,0.173,0.050,0.103,0.068,0.076,0.360,0.031,0.132,0.038,0.037,0.097,0.075,0.123,0.072,0.053,0.059,0.023,0.135,0.143,0.112,0.101,0.163,0.154,0.078,0.036,0.076,0.092,0.042,0.121,0.179,0.117,0.060,0.036,0.055,0.116,0.089,0.083,0.113,0.105,0.130
2,0.085,0.013,0.073,0.002,0.016,0.051,0.029,0.033,0.097,0.046,0.116,0.018,0.071,0.096,0.159,0.122,0.043,0.141,0.097,0.064,0.055,0.054,0.051,0.119,0.109,0.097,0.056,0.086,0.126,0.036,0.113,0.079,0.027,0.170,0.072,0.044,0.163,0.201,0.053,0.163,0.041,0.071,0.046,0.256,0.173,0.199,0.122,0.028,0.084,0.026,...,0.045,0.081,0.115,0.063,0.070,0.053,0.078,0.036,0.083,0.111,0.084,0.053,0.084,0.071,0.087,0.471,0.024,0.079,0.047,0.036,0.078,0.059,0.137,0.084,0.083,0.056,0.036,0.106,0.160,0.108,0.184,0.162,0.124,0.135,0.053,0.073,0.108,0.056,0.100,0.224,0.147,0.053,0.075,0.032,0.131,0.055,0.084,0.153,0.119,0.104
3,0.085,0.043,0.067,0.002,0.015,0.015,0.031,0.092,0.048,0.067,0.102,0.031,0.128,0.082,0.104,0.141,0.031,0.103,0.075,0.082,0.046,0.016,0.046,0.078,0.070,0.073,0.051,0.085,0.098,0.048,0.145,0.078,0.021,0.142,0.062,0.034,0.149,0.194,0.038,0.065,0.043,0.070,0.043,0.274,0.047,0.108,0.162,0.034,0.069,0.038,...,0.039,0.086,0.068,0.035,0.064,0.061,0.065,0.022,0.066,0.087,0.080,0.047,0.079,0.066,0.029,0.187,0.030,0.059,0.055,0.035,0.080,0.074,0.090,0.066,0.071,0.062,0.034,0.114,0.121,0.108,0.051,0.170,0.068,0.072,0.074,0.078,0.101,0.080,0.084,0.178,0.152,0.047,0.064,0.022,0.104,0.057,0.087,0.149,0.102,0.093
4,0.084,0.033,0.051,0.003,0.022,0.052,0.052,0.030,0.041,0.045,0.102,0.032,0.122,0.069,0.089,0.103,0.011,0.094,0.084,0.053,0.047,0.022,0.048,0.040,0.077,0.089,0.041,0.092,0.062,0.045,0.109,0.109,0.033,0.084,0.069,0.035,0.225,0.199,0.078,

In [22]:
startDate = datetime(2018,5,1)
endDate = datetime(2019,5,1)
def datetime_range(start, end, delta):
    current = start
    while current < end:
        yield current
        current += delta

dts = [dt.strftime('%Y-%m-%d T%H:%M Z') for dt in 
       datetime_range(startDate, endDate, 
       timedelta(minutes=15))]

In [24]:
in_dict = []
for uid in ogDatas:
    household_dict = {}
    household_dict["uid"] = uid
    
    count_range = range(len(ogDatas[uid]))
    
    ts_dict_list = []
    
    for count in count_range:
        ts_dict = {
            "time": dts[count],
            "power": ogDatas[uid][count]
        }
        ts_dict_list.append(ts_dict)
    
    household_dict['timeslot'] = ts_dict_list
    
    print(household_dict['uid'],household_dict['timeslot'][0], household_dict['timeslot'][35039])
    
    in_dict.append(household_dict)

아파트1-104-1206 {'time': '2018-05-01 T00:00 Z', 'power': 0.085} {'time': '2019-04-30 T23:45 Z', 'power': 0.014}
아파트1-104-303 {'time': '2018-05-01 T00:00 Z', 'power': 0.016} {'time': '2019-04-30 T23:45 Z', 'power': 0.033}
아파트1-104-1307 {'time': '2018-05-01 T00:00 Z', 'power': 0.052} {'time': '2019-04-30 T23:45 Z', 'power': 0.009}
아파트1-104-1208 {'time': '2018-05-01 T00:00 Z', 'power': 0.003} {'time': '2019-04-30 T23:45 Z', 'power': 0.053}
아파트1-104-408 {'time': '2018-05-01 T00:00 Z', 'power': 0.032} {'time': '2019-04-30 T23:45 Z', 'power': 0.025}
아파트1-104-203 {'time': '2018-05-01 T00:00 Z', 'power': 0.074} {'time': '2019-04-30 T23:45 Z', 'power': 0.06}
아파트1-103-1402 {'time': '2018-05-01 T00:00 Z', 'power': 0.053} {'time': '2019-04-30 T23:45 Z', 'power': 0.049}
아파트1-103-402 {'time': '2018-05-01 T00:00 Z', 'power': 0.096} {'time': '2019-04-30 T23:45 Z', 'power': 0.187}
아파트1-103-1201 {'time': '2018-05-01 T00:00 Z', 'power': 0.105} {'time': '2019-04-30 T23:45 Z', 'power': 0.044}
아파트1-103-801 {'

아파트1-104-204 {'time': '2018-05-01 T00:00 Z', 'power': 0.054} {'time': '2019-04-30 T23:45 Z', 'power': 0.02}
아파트1-104-1207 {'time': '2018-05-01 T00:00 Z', 'power': 0.064} {'time': '2019-04-30 T23:45 Z', 'power': 0.051}
아파트1-103-602 {'time': '2018-05-01 T00:00 Z', 'power': 0.095} {'time': '2019-04-30 T23:45 Z', 'power': 0.073}
아파트1-103-1302 {'time': '2018-05-01 T00:00 Z', 'power': 0.044} {'time': '2019-04-30 T23:45 Z', 'power': 0.009}
아파트1-103-1002 {'time': '2018-05-01 T00:00 Z', 'power': 0.149} {'time': '2019-04-30 T23:45 Z', 'power': 0.206}
아파트1-102-404 {'time': '2018-05-01 T00:00 Z', 'power': 0.043} {'time': '2019-04-30 T23:45 Z', 'power': 0.071}
아파트1-103-1102 {'time': '2018-05-01 T00:00 Z', 'power': 0.033} {'time': '2019-04-30 T23:45 Z', 'power': 0.127}
아파트1-102-1004 {'time': '2018-05-01 T00:00 Z', 'power': 0.047} {'time': '2019-04-30 T23:45 Z', 'power': 0.056}
아파트1-102-1402 {'time': '2018-05-01 T00:00 Z', 'power': 0.08} {'time': '2019-04-30 T23:45 Z', 'power': 0.127}
아파트1-101-301 {'

아파트2-2-902 {'time': '2018-05-01 T00:00 Z', 'power': 0.0} {'time': '2019-04-30 T23:45 Z', 'power': 0.12}
아파트2-2-1001 {'time': '2018-05-01 T00:00 Z', 'power': 0.0} {'time': '2019-04-30 T23:45 Z', 'power': 0.02}
아파트2-2-1508 {'time': '2018-05-01 T00:00 Z', 'power': 0.0} {'time': '2019-04-30 T23:45 Z', 'power': 0.043}
아파트2-3-601 {'time': '2018-05-01 T00:00 Z', 'power': 0.0} {'time': '2019-04-30 T23:45 Z', 'power': 0.073}
아파트2-3-1002 {'time': '2018-05-01 T00:00 Z', 'power': 0.0} {'time': '2019-04-30 T23:45 Z', 'power': 0.04}
아파트2-4-205 {'time': '2018-05-01 T00:00 Z', 'power': 0.0} {'time': '2019-04-30 T23:45 Z', 'power': 0.1}
아파트2-4-405 {'time': '2018-05-01 T00:00 Z', 'power': 0.0} {'time': '2019-04-30 T23:45 Z', 'power': 0.1}
아파트2-4-408 {'time': '2018-05-01 T00:00 Z', 'power': 0.0} {'time': '2019-04-30 T23:45 Z', 'power': 0.047}
아파트2-4-509 {'time': '2018-05-01 T00:00 Z', 'power': 0.0} {'time': '2019-04-30 T23:45 Z', 'power': 0.09}
아파트2-4-908 {'time': '2018-05-01 T00:00 Z', 'power': 0.0} {'t

아파트3-103-1302 {'time': '2018-05-01 T00:00 Z', 'power': 0.051} {'time': '2019-04-30 T23:45 Z', 'power': 0.037}
아파트3-103-310 {'time': '2018-05-01 T00:00 Z', 'power': 0.053} {'time': '2019-04-30 T23:45 Z', 'power': 0.072}
아파트3-101-806 {'time': '2018-05-01 T00:00 Z', 'power': 0.069} {'time': '2019-04-30 T23:45 Z', 'power': 0.061}
아파트3-101-506 {'time': '2018-05-01 T00:00 Z', 'power': 0.051} {'time': '2019-04-30 T23:45 Z', 'power': 0.097}
아파트3-103-803 {'time': '2018-05-01 T00:00 Z', 'power': 0.042} {'time': '2019-04-30 T23:45 Z', 'power': 0.038}
아파트3-101-405 {'time': '2018-05-01 T00:00 Z', 'power': 0.034} {'time': '2019-04-30 T23:45 Z', 'power': 0.104}
아파트3-103-211 {'time': '2018-05-01 T00:00 Z', 'power': 0.055} {'time': '2019-04-30 T23:45 Z', 'power': 0.058}
아파트3-101-1001 {'time': '2018-05-01 T00:00 Z', 'power': 0.176} {'time': '2019-04-30 T23:45 Z', 'power': 0.256}
아파트3-103-623 {'time': '2018-05-01 T00:00 Z', 'power': 0.058} {'time': '2019-04-30 T23:45 Z', 'power': 0.061}
아파트3-101-102 {'ti

아파트3-103-1018 {'time': '2018-05-01 T00:00 Z', 'power': 0.052} {'time': '2019-04-30 T23:45 Z', 'power': 0.049}
아파트3-103-213 {'time': '2018-05-01 T00:00 Z', 'power': 0.057} {'time': '2019-04-30 T23:45 Z', 'power': 0.168}
아파트3-103-1019 {'time': '2018-05-01 T00:00 Z', 'power': 0.013} {'time': '2019-04-30 T23:45 Z', 'power': 0.027}
아파트3-103-701 {'time': '2018-05-01 T00:00 Z', 'power': 0.102} {'time': '2019-04-30 T23:45 Z', 'power': 0.093}
아파트3-103-1007 {'time': '2018-05-01 T00:00 Z', 'power': 0.109} {'time': '2019-04-30 T23:45 Z', 'power': 0.076}
아파트3-103-1322 {'time': '2018-05-01 T00:00 Z', 'power': 0.044} {'time': '2019-04-30 T23:45 Z', 'power': 0.031}
아파트3-103-915 {'time': '2018-05-01 T00:00 Z', 'power': 0.031} {'time': '2019-04-30 T23:45 Z', 'power': 0.058}
아파트3-103-822 {'time': '2018-05-01 T00:00 Z', 'power': 0.039} {'time': '2019-04-30 T23:45 Z', 'power': 0.037}
아파트3-103-911 {'time': '2018-05-01 T00:00 Z', 'power': 0.089} {'time': '2019-04-30 T23:45 Z', 'power': 0.09}
아파트3-101-1302 {'

아파트4-103-407 {'time': '2018-05-01 T00:00 Z', 'power': 0.088} {'time': '2019-04-30 T23:45 Z', 'power': 0.08}
아파트4-103-1206 {'time': '2018-05-01 T00:00 Z', 'power': 0.117} {'time': '2019-04-30 T23:45 Z', 'power': 0.154}
아파트4-103-108 {'time': '2018-05-01 T00:00 Z', 'power': 0.161} {'time': '2019-04-30 T23:45 Z', 'power': 0.133}
아파트4-105-102 {'time': '2018-05-01 T00:00 Z', 'power': 0.182} {'time': '2019-04-30 T23:45 Z', 'power': 0.101}
아파트4-103-1102 {'time': '2018-05-01 T00:00 Z', 'power': 0.044} {'time': '2019-04-30 T23:45 Z', 'power': 0.067}
아파트4-105-601 {'time': '2018-05-01 T00:00 Z', 'power': 0.051} {'time': '2019-04-30 T23:45 Z', 'power': 0.067}
아파트4-104-1501 {'time': '2018-05-01 T00:00 Z', 'power': 0.057} {'time': '2019-04-30 T23:45 Z', 'power': 0.038}
아파트4-103-702 {'time': '2018-05-01 T00:00 Z', 'power': 0.126} {'time': '2019-04-30 T23:45 Z', 'power': 0.11}
아파트4-102-104 {'time': '2018-05-01 T00:00 Z', 'power': 0.094} {'time': '2019-04-30 T23:45 Z', 'power': 0.025}
아파트4-103-902 {'tim

In [18]:
# Mongo 연결
from pymongo import MongoClient as mc

mongo_uri = "mongodb://localhost:27017"
client = mc(mongo_uri)

keti_pr_db = client.keti_pattern_recognition
keti_pr_db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'keti_pattern_recognition')

In [21]:
coln = keti_pr_db.household_info

coln

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'keti_pattern_recognition'), 'household_info')

In [25]:
coln.insert_many(in_dict)

In [30]:
coln.find_one({"uid": "아파트3-102-701"})

{'_id': ObjectId('60ef9f4eb51f3d7630aa2531'),
 'uid': '아파트3-102-701',
 'timeslot': [{'time': '2018-05-01 T00:00 Z', 'power': 0.093},
  {'time': '2018-05-01 T00:15 Z', 'power': 0.154},
  {'time': '2018-05-01 T00:30 Z', 'power': 0.101},
  {'time': '2018-05-01 T00:45 Z', 'power': 0.076},
  {'time': '2018-05-01 T01:00 Z', 'power': 0.075},
  {'time': '2018-05-01 T01:15 Z', 'power': 0.085},
  {'time': '2018-05-01 T01:30 Z', 'power': 0.092},
  {'time': '2018-05-01 T01:45 Z', 'power': 0.061},
  {'time': '2018-05-01 T02:00 Z', 'power': 0.09},
  {'time': '2018-05-01 T02:15 Z', 'power': 0.066},
  {'time': '2018-05-01 T02:30 Z', 'power': 0.093},
  {'time': '2018-05-01 T02:45 Z', 'power': 0.07},
  {'time': '2018-05-01 T03:00 Z', 'power': 0.073},
  {'time': '2018-05-01 T03:15 Z', 'power': 0.054},
  {'time': '2018-05-01 T03:30 Z', 'power': 0.043},
  {'time': '2018-05-01 T03:45 Z', 'power': 0.063},
  {'time': '2018-05-01 T04:00 Z', 'power': 0.04},
  {'time': '2018-05-01 T04:15 Z', 'power': 0.062},
  {

In [19]:
client.close()